# 05 Recommender Systems

- Content-based recommender systems
- Collaborative Filtering

<p align="center"> 
<img src="../images/05_Recommender/what_are_recommender_systems.png", width=800, height=600>|
</p>

<p align="center"> 
<img src="../images/05_Recommender/types_recommender_systems.png", width=800, height=600>
</p>

<p align="center"> 
<img src="../images/05_Recommender/pros_recommender_systems.png", width=800, height=600>
</p>

<p align="center"> 
<img src="../images/05_Recommender/implement_recommender_systems.png", width=800, height=600>
</p>

<p align="center"> 
<img src="../images/05_Recommender/applications_recommender_systems.png", width=800, height=600>
</p>

# Content-based recommender systems

<p align="center"> 
<img src="../images/05_Recommender/what_is_content_based_recsys.png", width=800, height=600>
</p>

<p align="center"> 
<img src="../images/05_Recommender/how_content_based_recsys_works.png", width=800, height=600>
</p>

<p align="center"> 
<img src="../images/05_Recommender/user_ratings_movies_matrix.png", width=800, height=600>
</p>

<p align="center"> 
<img src="../images/05_Recommender/user_profile.png", width=800, height=600>
</p>

<p align="center"> 
<img src="../images/05_Recommender/candidate_movies.png", width=800, height=600>
</p>

<p align="center"> 
<img src="../images/05_Recommender/recommendation_matrix.png", width=800, height=600>
</p>

<p align="center"> 
<img src="../images/05_Recommender/predicted_ratings_for_user.png", width=800, height=600>
</p>

<p align="center"> 
<img src="../images/05_Recommender/other_genre_problem.png", width=800, height=600>
</p>

# Collaborative Filtering